# DEBUG

In [0]:
 import datetime as dt
 from zoneinfo import ZoneInfo

 from alpaca_pyspark.bars import HistoricalBarsReader

In [0]:
tz = ZoneInfo("America/New_York")

hist_bars_opts = {
    "symbols": ["AAPL", "MSFT", "GOOG"],
    'APCA-API-KEY-ID': dbutils.secrets.get(scope="alpaca_paper_api", key="ALPACA_API_KEY"),
    'APCA-API-SECRET-KEY': dbutils.secrets.get(scope="alpaca_paper_api", key="ALPACA_SECRET_KEY"),
    "timeframe": "1Day",
    "start": dt.datetime(2021, 1, 1, tzinfo=tz).isoformat(),
    "end": dt.datetime(2022, 1, 1, tzinfo=tz).isoformat(),
    "limit": 1000
}

In [0]:
bars_schema = """
    symbol: STRING,
    time: TIMESTAMP,
    open: FLOAT,
    high: FLOAT,
    low: FLOAT,
    close: FLOAT,
    volume: INT,
    trade_count: INT,
    vwap: FLOAT
"""

hist_bars_reader = HistoricalBarsReader(bars_schema, hist_bars_opts)

In [0]:
sym_parts = hist_bars_reader.partitions()

In [0]:
sym_parts[0].symbol

In [0]:
bars_iter = hist_bars_reader.read(sym_parts[0])

In [0]:
for bar in bars_iter:
    print(bar)

In [0]:
test_bars = [
    ('AAPL', '2021-01-04T05:00:00Z', 133.52, 133.6116, 126.76, 129.41, 158211374, 1310229, 129.717982),
    ('AAPL', '2021-01-05T05:00:00Z', 128.89, 131.74, 128.43, 131.01, 105863439, 707583, 130.738233),
    ('AAPL', '2021-01-06T05:00:00Z', 127.72, 131.0499, 126.382, 126.66, 165568781, 1202579, 128.249403),
    ('AAPL', '2021-01-07T05:00:00Z', 128.36, 131.63, 127.86, 130.92, 118743769, 718362, 130.185457),
    ('AAPL', '2021-01-08T05:00:00Z', 132.43, 132.63, 130.23, 132.05, 112696090, 798393, 131.580087)
]
for bar in test_bars:
    print(bar)

In [0]:
import requests

headers = {
    'Content-Type': 'application/json',
    'APCA-API-KEY-ID': dbutils.secrets.get(scope="alpaca_paper_api", key="ALPACA_API_KEY"),
    'APCA-API-SECRET-KEY': dbutils.secrets.get(scope="alpaca_paper_api", key="ALPACA_SECRET_KEY")
}

test_uri = "https://data.alpaca.markets/v2/stocks/bars?timeframe=1Day&start=2021-01-01T00%3A00%3A00-05%3A00&end=2022-01-01T00%3A00%3A00-05%3A00&limit=1000&symbols=AAPL"

In [0]:
test_uri

In [0]:
resp = requests.get(test_uri, headers=headers)
resp.status_code

In [0]:
resp.raise_for_status()

In [0]:
pg = resp.json()

In [0]:
sym = 'AAPL'
bar = pg['bars'][sym][1:5]

In [0]:
bar

In [0]:
bar_cols = [
    "t", #timestamp
    "o", #open
    "h", #high
    "l", #low
    "c", #close
    "v", #volume
    "n", #trade_count
    "vw" #vwap
]

In [0]:
tuple(bar[col_key] for col_key in bar_cols)

In [0]:
for bar in pg['bars'][sym][:5]:
    print((sym,) + tuple(bar[col_key] for col_key in bar_cols))